# Create the training mapping sheet

In [1]:
import os
import sys
import subprocess
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from deep_translator import GoogleTranslator

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

clean_data_path = os.path.join(parent_dir, "Data", "Clean_data", "Biotheus", "Training_Nantong")

translator = GoogleTranslator(source='zh-CN', target='en')


Load the cleaned data

In [2]:
data_list = ['DP', 'DS', 'EHS', 'IT', 'MM', 'MSAT', 'PM', 'QA', 'QC', 'ASAT', 'CS', 'QM']
data_dict = {}
for name in data_list:
    file_path = os.path.join(clean_data_path, f"{name}.xlsx")
    data_dict[name] = pd.read_excel(file_path)


# 1. Learner Role

In [3]:
columns = ["Name", "Unit", "Scope", "RegArea", "Description"]
LR = pd.DataFrame(columns=columns)
LR

,Name,Unit,Scope,RegArea,Description


In [4]:
# Combine 'Learner Role' columns from all dataframes into one column
all_roles = []
for key in data_dict.keys():
    roles = pd.DataFrame(data_dict[key].columns[2:], columns=['Learner Role'])
    all_roles.append(roles)

combined_roles = pd.concat(all_roles, ignore_index=True)

LR_Translation = [translator.translate(item) for item in combined_roles["Learner Role"]]

LR["Name"] = combined_roles["Learner Role"]
LR["Unit"] = "1009" # Nantong 
LR["Scope"] = "Department"
LR["RegArea"] = "GMP"
LR["Description"] = LR_Translation
LR

,Name,Unit,Scope,RegArea,Description
0,DP总监,1009,Department,GMP,DP Director
1,DP经理,1009,Department,GMP,DP Manager
2,DP主管,1009,Department,GMP,DP supervisor
3,配液及清洗灭菌,1009,Department,GMP,"Liquid preparation, cleaning and sterilization"
4,洗烘瓶,1009,Department,GMP,Washing and drying bottles
...,...,...,...,...,...
78,DS清洁人员,1009,Department,GMP,DS cleaning staff
79,DP清洁人员,1009,Department,GMP,DP cleaning staff
80,PM清洁人员,1009,Department,GMP,PM cleaning staff
81,MM清洁人员,1009,Department,GMP,MM cleaning staff


# 2. Curriculum

In [5]:
columns = ["Name", "Unit", "Owning Department", "RegArea", "Scope", "Description"]
Curriculum = pd.DataFrame(columns=columns)
Curriculum


,Name,Unit,Owning Department,RegArea,Scope,Description


LR = Curriculum

In [6]:
all_cur = []
all_dep = []

for key in data_dict.keys():
    roles = pd.DataFrame(data_dict[key].columns[2:], columns=['Name'])  # rename column to match your Curriculum
    all_cur.append(roles)
    
    # create a DataFrame of same length as roles, filled with the department name
    dept_df = pd.DataFrame([key] * len(roles), columns=['Owning Department'])
    all_dep.append(dept_df)

combined_cur = pd.concat(all_cur, ignore_index=True)
combined_dep = pd.concat(all_dep, ignore_index=True)

Curriculum_Translation = [translator.translate(item) for item in combined_cur["Name"]]

Curriculum["Name"] = combined_cur["Name"]
Curriculum["Owning Department"] = combined_dep["Owning Department"]
Curriculum["Unit"] = "1009"  # Nantong
Curriculum["Scope"] = "Department"  # optional, as in your example
Curriculum["RegArea"] = "GMP"
Curriculum["Description"] = Curriculum_Translation

Curriculum


,Name,Unit,Owning Department,RegArea,Scope,Description
0,DP总监,1009,DP,GMP,Department,DP Director
1,DP经理,1009,DP,GMP,Department,DP Manager
2,DP主管,1009,DP,GMP,Department,DP supervisor
3,配液及清洗灭菌,1009,DP,GMP,Department,"Liquid preparation, cleaning and sterilization"
4,洗烘瓶,1009,DP,GMP,Department,Washing and drying bottles
...,...,...,...,...,...,...
78,DS清洁人员,1009,CS,GMP,Department,DS cleaning staff
79,DP清洁人员,1009,CS,GMP,Department,DP cleaning staff
80,PM清洁人员,1009,CS,GMP,Department,PM cleaning staff
81,MM清洁人员,1009,CS,GMP,Department,MM cleaning staff


# 3. Training Requirement

In [7]:
columns = ["Name", "Unit", "Training Requirement Type", "Required Quiz", "Duration", "Duration Unit", "Estimated Time to Complete (Minutes)", "Recur", "Recur Frequency", "Description"]
TR = pd.DataFrame(columns=columns)
TR

,Name,Unit,Training Requirement Type,Required Quiz,Duration,Duration Unit,Estimated Time to Complete (Minutes),Recur,Recur Frequency,Description


In [8]:
data_list = ['DP', 'DS', 'EHS', 'IT', 'MM', 'MSAT', 'PM', 'QA', 'QC', 'ASAT', 'CS', 'QM']

# List to store all doc info
all_docs = []

for key in data_list:
    df = data_dict[key]
    
    # Select only the relevant columns
    docs = df[['doc number', 'doc name']].copy()
    
    all_docs.append(docs)

# Concatenate all and drop duplicates
combined_docs = pd.concat(all_docs, ignore_index=True).drop_duplicates()

# Optional: reset index
combined_docs.reset_index(drop=True, inplace=True)

# TR["Name"] = combined_docs["doc number"] + " - " + combined_docs["doc name"]
TR["Name"] = combined_docs["doc name"]
TR["Unit"] = "1009"  # Nantong
TR["Required Quiz"] = "No"
TR["Duration"] = "30"
TR["Duration Unit"] = "Days"
# TR["Estimated Time to Complete"] = ?
TR["Recur"] = "No"
TR["Training Requirement Type"] = "Vault Document Training Requirement"
TR["Estimated Time to Complete (Minutes)"] = "120"
TR

,Name,Unit,Training Requirement Type,Required Quiz,Duration,Duration Unit,Estimated Time to Complete (Minutes),Recur,Recur Frequency,Description
0,质量文件管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
1,质量文件编号管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
2,良好书写实践管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
3,GMP培训管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
4,记录管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
543,DP01车间配制与过滤岗位标准操作规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
544,DP01车间物品装配指南,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
545,DP01车间制剂灭菌柜装载与灭菌标准操作规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
546,DP01车间GMP洗衣房灭菌柜装载与灭菌标准操作规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN


Need to update the table using the annual training plan

In [9]:
path = os.path.join(clean_data_path, "Annual_Training_Plan.xlsx")
Annual_Training_Plan = pd.read_excel(path)

freq_map = (
    Annual_Training_Plan
        .groupby('Training topic')['Training frequency (month)']
        .max()
)
'''
Because one topic appears multiple times, this:
- collapses them into one row per topic
- keeps the largest frequency value
'''

# Create a mask for rows in TR where 'Name' is in freq_map index
mask = TR['Name'].isin(freq_map.index) # -> True/False for each row

TR.loc[mask, 'Recur'] = 'Yes'
TR.loc[mask, 'Recur Frequency'] = TR.loc[mask, 'Name'].map(freq_map)


TR[TR['Name'].isin(Annual_Training_Plan['Training topic'])]
TR


,Name,Unit,Training Requirement Type,Required Quiz,Duration,Duration Unit,Estimated Time to Complete (Minutes),Recur,Recur Frequency,Description
0,质量文件管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
1,质量文件编号管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
2,良好书写实践管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
3,GMP培训管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
4,记录管理规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
543,DP01车间配制与过滤岗位标准操作规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
544,DP01车间物品装配指南,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
545,DP01车间制剂灭菌柜装载与灭菌标准操作规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN
546,DP01车间GMP洗衣房灭菌柜装载与灭菌标准操作规程,1009,Vault Document Training Requirement,No,30,Days,120,No,NaN,NaN


# 4. LINK C-LR

In [10]:
columns = ["Curriculum", "Learner Role Name"]
link_C_LR = pd.DataFrame(columns=columns)
link_C_LR

,Curriculum,Learner Role Name


In [11]:
link_C_LR["Curriculum"] = LR["Name"]
link_C_LR["Learner Role Name"] = LR["Name"]
link_C_LR

,Curriculum,Learner Role Name
0,DP总监,DP总监
1,DP经理,DP经理
2,DP主管,DP主管
3,配液及清洗灭菌,配液及清洗灭菌
4,洗烘瓶,洗烘瓶
...,...,...
78,DS清洁人员,DS清洁人员
79,DP清洁人员,DP清洁人员
80,PM清洁人员,PM清洁人员
81,MM清洁人员,MM清洁人员


# 5. Link TR-C

In [12]:
data_list = ['DP', 'DS', 'EHS', 'IT', 'MM', 'MSAT', 'PM', 'QA', 'QC', 'ASAT', 'CS', 'QM']

link_tr_c_rows = []

for key in data_list:

    df = data_dict[key]

    col_num = len(df.columns) - 2  # Subtract 2 for 'doc number' and 'doc name'

    for i in range(col_num):
        learner_role = df.columns[i + 2]

        for j in range(len(df)):
            training_req = f"{df.at[j, 'doc name']}"
            
            if df.at[j, learner_role] == 1:
                link_tr_c_rows.append({
                    "Curriculum": learner_role,
                    "Training Requirement": training_req
                })

link_TR_C = pd.DataFrame(link_tr_c_rows)
link_TR_C

,Curriculum,Training Requirement
0,DP总监,质量文件管理规程
1,DP总监,良好书写实践管理规程
2,DP总监,GMP培训管理规程
3,DP总监,记录管理规程
4,DP总监,签名授权管理规程
...,...,...
7780,质量负责人,污染控制策略
7781,质量负责人,无菌验证管理规程
7782,质量负责人,数据完整性
7783,质量负责人,偏差及纠正预防措施管理规程


# Combine all tabs

In [13]:
Path = os.path.join(clean_data_path, "Training.xlsx")
with pd.ExcelWriter(Path) as writer:
    LR.to_excel(writer, sheet_name="Learner Role", index=False)
    Curriculum.to_excel(writer, sheet_name="Curriculum", index=False)
    TR.to_excel(writer, sheet_name="Training Requirement", index=False)
    link_C_LR.to_excel(writer, sheet_name="Link C-LR", index=False)
    link_TR_C.to_excel(writer, sheet_name="Link TR-C", index=False)